In [ ]:
import torch
import torch.nn as nn

In [ ]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = """In a cozy little cottage nestled on the hillside, a family of mice spent their days
            gathering crumbs and telling tales of daring adventures in the wide world beyond. Every evening,
            they huddled around the fireplace, where Grandpa Mouse would spin yarns of heroic quests and legendary
            cheese heists. Little Timmy Mouse, with wide-eyed wonder, listened intently, dreaming of the day he'd
            embark on his very own mouse-sized journey.""".split()

In [ ]:
# By deriving a set from `raw_text`, we duplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)} #make dictonort word and index assign
ix_to_word = {ix:word for ix, word in enumerate(vocab)} #made dic index and word

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target)) #data=data+context

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__() #

        #out: 1 x emdedding_dim for CNN
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)


    def forward(self, inputs): #How to forward pass
        embeds = sum(self.embeddings(inputs)).view(1,-1) #normalize 1,-1
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word): #word milali ki index prints
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [ ]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()  #negative Log Likelihood Loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [ ]:
# Training (for loop)
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    # optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [ ]:
# Testing
context = ['where', 'Grandpa', 'would','spin']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: In a cozy little cottage nestled on the hillside, a family of mice spent their days gathering crumbs and telling tales of daring adventures in the wide world beyond. Every evening, they huddled around the fireplace, where Grandpa Mouse would spin yarns of heroic quests and legendary cheese heists. Little Timmy Mouse, with wide-eyed wonder, listened intently, dreaming of the day he'd embark on his very own mouse-sized journey.

Context: ['where', 'Grandpa', 'would', 'spin']

Prediction: Mouse
